# 個人課題1

- Github上でGoogleが管理しているリポジトリの情報をスクレイピングして下記の情報を取得する
    - リポジトリ名
    - 主要な言語
    - スターの数
- 上記のデータを保存するためのDBを作成し，スクレイピングしたデータを保存する
    - GithubのAPIは使わないこと（スクレイピングの課題なので）
    - ちゃんと`time.sleep(1)`入れてね
- 保存したデータをSELECT文で表示する
- ソースコードはGithubフローに従って管理すること
    1. ブランチ作成（add-exercise-1）
    2. ステージング
    3. コミット（コミットはキリのいいところでこまめにすること）
    4. プッシュ
    5. プルリク
    6. コードレビュー，マージ（プルリクは自分で処理してよい）
- GoogleクラスルームでリポジトリURLを提出
- 提出期限：11月25日（火）23時59分

## ----------------------------------------------------------

## ライブラリのインポート

In [4]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time

## SQLiteデータベースの初期化

In [5]:
# DB接続
conn = sqlite3.connect("google_repos.db")
cur = conn.cursor()

# テーブル作成
cur.execute("""
CREATE TABLE IF NOT EXISTS repos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    language TEXT,
    stars INTEGER
)
""")
conn.commit()

## スクレイピング関数

In [6]:
BASE_URL = "https://github.com/google?tab=repositories&page="

def scrape_page(page_num):
    url = BASE_URL + str(page_num)
    print(f"Scraping: {url}")
    try:
        res = requests.get(url)
        res.raise_for_status()
        soup = BeautifulSoup(res.text, "html.parser")
        repo_list = soup.find_all("li", class_="Box-row")

        results = []
        for repo in repo_list:
            # リポジトリ名
            name_tag = repo.find("a", itemprop="name codeRepository")
            name = name_tag.text.strip() if name_tag else "Unknown"

            # 言語
            lang_tag = repo.find("span", itemprop="programmingLanguage")
            language = lang_tag.text.strip() if lang_tag else "Unknown"

            # スター数
            star_tag = repo.find("a", href=lambda x: x and "stargazers" in x)
            if star_tag:
                stars_text = star_tag.text.strip().replace(",", "")
                try:
                    stars = int(stars_text)
                except:
                    stars = 0
            else:
                stars = 0

            results.append((name, language, stars))

        return results

    except Exception as e:
        print("Error:", e)
        return []

## データ取得 & DB保存

In [7]:
page = 1
while True:
    repos = scrape_page(page)
    if not repos:  # データが取得できなければ終了
        break

    # DBに保存
    for r in repos:
        cur.execute(
            "INSERT INTO repos (name, language, stars) VALUES (?, ?, ?)",
            (r[0], r[1], r[2])
        )
    conn.commit()

    time.sleep(1)
    page += 1

Scraping: https://github.com/google?tab=repositories&page=1
Scraping: https://github.com/google?tab=repositories&page=2
Scraping: https://github.com/google?tab=repositories&page=3
Scraping: https://github.com/google?tab=repositories&page=4
Scraping: https://github.com/google?tab=repositories&page=5
Scraping: https://github.com/google?tab=repositories&page=6
Scraping: https://github.com/google?tab=repositories&page=7
Scraping: https://github.com/google?tab=repositories&page=8
Scraping: https://github.com/google?tab=repositories&page=9
Scraping: https://github.com/google?tab=repositories&page=10
Scraping: https://github.com/google?tab=repositories&page=11
Scraping: https://github.com/google?tab=repositories&page=12
Scraping: https://github.com/google?tab=repositories&page=13
Scraping: https://github.com/google?tab=repositories&page=14
Scraping: https://github.com/google?tab=repositories&page=15
Scraping: https://github.com/google?tab=repositories&page=16
Scraping: https://github.com/goog

KeyboardInterrupt: 

## 保存データの確認

In [8]:
print("保存済みデータ一覧:")
cur.execute("SELECT id, name, language, stars FROM repos")
for row in cur.fetchall():
    print(row)

conn.close()

保存済みデータ一覧:
(1, 'perfetto', 'C++', 5022)
(2, 'adk-java', 'Java', 973)
(3, 'ksp', 'Kotlin', 3307)
(4, 'XNNPACK', 'C', 2181)
(5, 'koladata', 'C++', 27)
(6, 'skia-buildbot', 'Go', 158)
(7, 'merchant-api-samples', 'Java', 10)
(8, 'xls', 'C++', 1375)
(9, 'clusterfuzz', 'Python', 5491)
(10, 'skia', 'C++', 10275)
(11, 'perfetto', 'C++', 5022)
(12, 'adk-java', 'Java', 973)
(13, 'ksp', 'Kotlin', 3307)
(14, 'XNNPACK', 'C', 2181)
(15, 'koladata', 'C++', 27)
(16, 'skia-buildbot', 'Go', 158)
(17, 'merchant-api-samples', 'Java', 10)
(18, 'xls', 'C++', 1375)
(19, 'clusterfuzz', 'Python', 5491)
(20, 'skia', 'C++', 10275)
(21, 'perfetto', 'C++', 5022)
(22, 'adk-java', 'Java', 973)
(23, 'ksp', 'Kotlin', 3307)
(24, 'XNNPACK', 'C', 2181)
(25, 'koladata', 'C++', 27)
(26, 'skia-buildbot', 'Go', 158)
(27, 'merchant-api-samples', 'Java', 10)
(28, 'xls', 'C++', 1375)
(29, 'clusterfuzz', 'Python', 5491)
(30, 'skia', 'C++', 10275)
(31, 'perfetto', 'C++', 5022)
(32, 'adk-java', 'Java', 973)
(33, 'ksp', 'Kotlin', 3

## 保存データの中身を確認

In [3]:
cur.execute("SELECT id, name, language, stars FROM repos LIMIT 10")
for row in cur.fetchall():
    print(row)

(1, 'perfetto', 'C++', 5022)
(2, 'adk-java', 'Java', 973)
(3, 'ksp', 'Kotlin', 3307)
(4, 'XNNPACK', 'C', 2181)
(5, 'koladata', 'C++', 27)
(6, 'skia-buildbot', 'Go', 158)
(7, 'merchant-api-samples', 'Java', 10)
(8, 'xls', 'C++', 1375)
(9, 'clusterfuzz', 'Python', 5491)
(10, 'skia', 'C++', 10275)
